### Setup

In [1]:
import sys
import os
import numpy as np

sys.path.append(os.path.abspath(".."))

from src.data_processing import (
    process_csv_to_numpy, 
    load_processed_data,
    save_processed_data,
    filter_k_core,
    split_train_test,
    add_time_features,
    standardize_ratings,
    perform_hypothesis_test,
    reindex_data
)

### Load data

In [2]:
PROCESSED_FOLDER = "../data/processed"
data, user_map, product_map = load_processed_data(PROCESSED_FOLDER)

### Check data

In [3]:
print(f"Total data rows: {len(data)}")

Total data rows: 2023070


In [4]:
print(f"Data before k-core filtering: {data.shape}")

Data before k-core filtering: (2023070, 4)


### Compute Data Sparsity

In [5]:
n_users = len(user_map)
n_products = len(product_map)
n_ratings = data.shape[0]

total_possible_ratings = n_users * n_products
sparsity = 1 - (n_ratings / total_possible_ratings)

print(f"Matrix Sparsity: {sparsity:.8f}")

Matrix Sparsity: 0.99999329


### K-Core Filtering
1. **Tổng quan**\
**K-Core Filtering** là một kỹ thuật tiền xử lý dữ liệu dựa trên Graph Theory. Trong bối cảnh hệ thống gợi ý, dữ liệu tương tác được xem như một đồ thị hai phía giữa Users và Products.\
Một đồ thị con được gọi là **k-core** nếu và chỉ nếu:
    - Mỗi người dùng trong đồ thị đó đã đánh giá ít nhất $k$ sản phẩm.
    - Mỗi sản phẩm trong đồ thị đó được đánh giá bởi ít nhất $k$ người dùng.
    - Điều kiện này phải thỏa mãn đồng thời và ổn định.
2. **Mục đích**
Bộ dữ liệu Amazon Ratings có đặc tính **Long-tail** và Độ thưa cực cao (>99.99%). Việc áp dụng K-Core là bắt buộc vì 3 lý do chính sau:
    - Giải quyết vấn đề Cold Start & Nhiễu
      - Hàng trăm nghìn người dùng chỉ đánh giá đúng 1 lần rồi rời đi.
      - Mô hình Matrix Factorization không thể học được bất kỳ "Sở thích ẩn" nào từ một người chỉ có 1 dữ liệu mẫu. Những dữ liệu này là nhiễu, gây tốn tài nguyên tính toán mà không đem lại giá trị dự đoán. 
    - Đảm bảo độ tin cậy cho việc Đánh giá
      - Chúng ta cần chia dữ liệu thành tập Train và Test (ví dụ tỷ lệ 80/20).
      - Nếu User chỉ có 1-2 rating, việc chia tập Test là bất khả thi hoặc thiếu tin cậy. Với K-Core, ta đảm bảo mỗi User có đủ dữ liệu lịch sử để vừa học vừa kiểm thử.
    - Tăng tốc độ hội tụ
      - Việc loại bỏ các vùng dữ liệu thưa giúp giảm kích thước ma trận đáng kể, thường giảm 40-60% kích thước gốc nhưng giữ lại hầu hết các tương tác giá trị, giúp thuật toán SGD hội tụ nhanh hơn.

In [6]:
cleaned_data = filter_k_core(data, k=5)

In [7]:
cleaned_data_reindex, user_map_reindex, product_map_reindex = reindex_data(
    cleaned_data, 
    user_map,
    product_map
)

In [8]:
print(f"Data after k-core filtering shape: {cleaned_data.shape}")

Data after k-core filtering shape: (198502, 4)


In [9]:
print(f"Data after k-core filtering: {cleaned_data[:5]}")

Data after k-core filtering: [[2.9300000e+02 1.1500000e+02 1.0000000e+00 1.3910400e+09]
 [3.0000000e+02 1.1500000e+02 3.0000000e+00 1.3977792e+09]
 [3.0200000e+02 1.1500000e+02 4.0000000e+00 1.3784256e+09]
 [3.1300000e+02 1.1500000e+02 2.0000000e+00 1.3864608e+09]
 [3.1400000e+02 1.1500000e+02 3.0000000e+00 1.3821408e+09]]


### Feature Engineering
Trích xuất 2 đặc trưng mới từ cột `Timestamp`:
1. `Time_weight` (0.2 - 1.0): Chuẩn hóa Min-Max, dùng để làm trọng số cho hàm mất mát, giúp mô hình ưu tiên học dữ liệu mới nhất.
2. `Year`: Chuyển đổi dữ liệu Unix Time sang năm, dùng cho trực quan hóa và kiểm định thống kê để so sánh xu hướng.

In [10]:
enhanced_data = add_time_features(cleaned_data_reindex) # [user_id, product_id, rating, timestamp, year, weight]

In [11]:
print(f"Data shape after feature addition: {enhanced_data.shape}")

Data shape after feature addition: (198502, 6)


In [12]:
print(f"Data after feature addition: {enhanced_data[:5]}")

Data after feature addition: [[1.50000000e+01 0.00000000e+00 1.00000000e+00 1.39104000e+09
  2.01410959e+03 9.68535262e-01]
 [1.60000000e+01 0.00000000e+00 3.00000000e+00 1.39777920e+09
  2.01432329e+03 9.82640145e-01]
 [1.70000000e+01 0.00000000e+00 4.00000000e+00 1.37842560e+09
  2.01370959e+03 9.42133816e-01]
 [1.80000000e+01 0.00000000e+00 2.00000000e+00 1.38646080e+09
  2.01396438e+03 9.58951175e-01]
 [1.90000000e+01 0.00000000e+00 3.00000000e+00 1.38214080e+09
  2.01382740e+03 9.49909584e-01]]


### Hypothesis testing
Thực hiện Z-test một phía để so sánh điểm đánh giá trung bình giữa hai năm cao điểm nhất (2013 và 2014). Mục đích để xác định xem sự gia tăng điểm đánh giá năm 2014 so với 2013 là có ý nghĩa thống kê hay chỉ là ngẫu nhiên.
Giả thuyết và đối thuyết:
  - $H_0$: $\mu_{2014} \le \mu_{2013}$ (Chất lượng đánh giá không tăng).
  - $H_1$: $\mu_{2014} > \mu_{2013}$ (Chất lượng đánh giá tăng thực sự).
  
**Ý nghĩa:** Nếu $Z_{score} > Z_{critical}$ (bác bỏ $H_0$), điều này khẳng định xu hướng hành vi người dùng đang thay đổi tích cực theo thời gian. Đây là cơ sở khoa học để áp dụng Time Decay trong mô hình.

In [13]:
perform_hypothesis_test(enhanced_data, year_a=2013, year_b=2014, confidence_level=0.95)

Performing Z-Test: Ratings in 2014 > 2013?
Hypothesis: H0: Mean_B <= Mean_A | H1: Mean_B > Mean_A
Confidence Level: 95.0%
   2013: Mean=4.1963, N=84706
   2014: Mean=4.2118, N=60945
   Z-Score: 2.5212
   Result: Reject H0. Statistically Significant.


### Standardization
Chuẩn hóa dữ liệu của cột **Rating**. Điều này giúp mô hình hội tụ nhanh và ổn định. Đồng thời cần lưu lại `mean_rating` và `std_rating` để unscaled cho sau này.

In [14]:
final_data, mean_rating, std_rating = standardize_ratings(enhanced_data)

In [15]:
print(f"Final data after standardization: {final_data[:5]}")

Final data after standardization: [[ 1.50000000e+01  0.00000000e+00 -2.73483207e+00  1.39104000e+09
   2.01410959e+03  9.68535262e-01]
 [ 1.60000000e+01  0.00000000e+00 -1.02041396e+00  1.39777920e+09
   2.01432329e+03  9.82640145e-01]
 [ 1.70000000e+01  0.00000000e+00 -1.63204913e-01  1.37842560e+09
   2.01370959e+03  9.42133816e-01]
 [ 1.80000000e+01  0.00000000e+00 -1.87762302e+00  1.38646080e+09
   2.01396438e+03  9.58951175e-01]
 [ 1.90000000e+01  0.00000000e+00 -1.02041396e+00  1.38214080e+09
   2.01382740e+03  9.49909584e-01]]


In [16]:
print(f"Mean rating: {mean_rating}, Std rating: {std_rating}")

Mean rating: 4.190391028805755, Std rating: 1.1665765757272069


### Split train/test set
Áp dụng phương pháp Chia theo thời gian thay vì chia ngẫu nhiên. Trước tiên phải sắp xếp toàn bộ dữ liệu theo trình tự thời gian theo `Timestamp`. Cắt 80% dữ liệu đầu tiên để làm train set, 20% còn lại làm test set. Đảm bảo hệ thống gợi ý luôn hoạt động theo quy tắc dùng dữ liệu của quá khứ để dự đoán tương lai.

In [17]:
train_data, test_data = split_train_test(final_data, train_ratio=0.8)

In [18]:
print(f"Train set shape: {train_data.shape}")
print(f"Test set shape: {test_data.shape}")

Train set shape: (158801, 6)
Test set shape: (39701, 6)


### Save data

In [19]:
save_processed_data(None, user_map_reindex, product_map_reindex, PROCESSED_FOLDER)

No data matrix to save.
Saved user and product maps to ../data/processed


In [20]:
mean_path = os.path.join(PROCESSED_FOLDER, "mean_rating.npy")
np.save(mean_path, np.array([mean_rating]))

std_path = os.path.join(PROCESSED_FOLDER, "std_rating.npy")
np.save(std_path, np.array([std_rating]))

In [21]:
train_path = os.path.join(PROCESSED_FOLDER, "train_data.npy")
np.save(train_path, train_data)

test_path = os.path.join(PROCESSED_FOLDER, "test_data.npy")
np.save(test_path, test_data)